<a href="https://colab.research.google.com/github/duttapratikcsc/LSTM_TextGeneration/blob/master/Bidrectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import sys
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Bidirectional
# from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from random import random
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

TensorFlow is already loaded. Please restart the runtime to change versions.


# Get List of Words from Corpus 

In [0]:
def get_Word(fp):
    word = []
    raw_text = open(fp).read().lower()
    raw_text = raw_text.split("\n")
    for row in raw_text:
        temp = row.split(" ")
        for i in range(len(temp)):
            word.append(temp[i])
    word.append("\n")
    print("Length of total word_list:-", len(word))
    return(word)

In [3]:
filename = 'class_3_p.csv'
seq = 3
total_words = get_Word(filename)

Length of total word_list:- 30874


Get Initial n-grams as well as last word for text generation

In [0]:
def get_initial_last_Word(fp, seq_no):
    initial_word = []
    last_word = []

    raw_text = open(fp).readlines()
    for row in raw_text:
        temp= row.split(" ")
        temp_word = ""
        for index in range(0, seq_no):
            temp_word += temp[0+index]+" "
        initial_word.append(temp_word.rstrip(" "))
        last_word.append(temp[-1].replace("\n",""))
    
    dami = list(set(initial_word))
    dx = [x.split(" ") for x in dami]
    return(dx, list(set(last_word)))

# Generate Word to Integer and Vice-versa for Embedding

In [0]:
#First create a set of all of the distinct words,
# then creating a map of each word to a unique integer.
def get_Word_dict(word_list):
    uniq_word = sorted(list(set(word_list)))
    uniqword_to_int = dict((c, i) for i, c in enumerate(uniq_word))
    print("Total no. Words:- ", len(word_list))
    print("Total no. of unique words:- ", len(uniq_word))
    return(uniqword_to_int)

def get_int_to_word(word_list):
    uniq_word = sorted(list(set(word_list)))
    int_to_word = dict((i, c) for i, c in enumerate(uniq_word))
    return(int_to_word)

In [7]:
word_dict = get_Word_dict(total_words)
int_to_word = get_int_to_word(total_words)
initial_word, last_word = get_initial_last_Word(filename, seq)
fileNN=open("initialWords.txt","w")
print(initial_word,file=fileNN)
fileNN.close()

Total no. Words:-  30874
Total no. of unique words:-  1011


# Pre Processing of Data
Prepare the dataset of input to output pairs encoded as integers

In [0]:
# prepare the dataset of input to output pairs encoded as integers
def Data_Preprocessing(fp, seq_no, word_dict):
    datax = []
    datay = []
    datay_word = []
    raw_text = open(fp).read().lower()
    raw_text = raw_text.split("\n")
    fdatax = open('data_input.csv', "w")
    fdatax_2 = open('data_input_string.csv', "w")
    for row in raw_text:
        temp = (row +" \n").split(" ")
        for i in range(len(temp)-seq_no):
            temp_word = str(temp[i])
            temp_word_int = str(word_dict[temp[i]])
            for index in range(1, seq_no+1):
                temp_word += ","+str(temp[i+index])
                temp_word_int += ","+str(word_dict[temp[i+index]])
            print(temp_word,file=fdatax_2)
            print(temp_word_int,file=fdatax)
    fdatax.close()
    fdatax_2.close()
    names = [str(i) for i in range(seq_no+1)]
    datax = pd.read_csv("data_input.csv",names=names)
    datax_2 = datax/len(word_dict)
    names = [str(i) for i in range(seq_no)]
    r = datax_2[names]
    s = datax[[str(seq_no)]] 
    return(r, s,datax)

In [9]:
datax, datay,total_dataset = Data_Preprocessing(filename, seq, word_dict)
no_patterns = len(datax)

print("datax: ", datax.shape)
print("datay: ", datay.shape)

datax:  (27600, 3)
datay:  (27600, 1)


In [10]:
X_train, X_val, y_train, y_val = train_test_split(datax,datay,test_size = 0.4)
print("X_train: ", X_train.shape)
print("X_val: ", X_val.shape)
print("Y_train: ", y_train.shape)
print("Y_val: ", y_val.shape)

X_train:  (16560, 3)
X_val:  (11040, 3)
Y_train:  (16560, 1)
Y_val:  (11040, 1)


Generate Caterogical One-hot Encodding for Output Layer

In [0]:
def create_categorical(y, total_no_words):
    categorical_data = np.zeros((len(y), total_no_words))
    print(len(y))
    for row in range(len(y)):
        temp = y[row]
        categorical_data[row][temp] = 1
    print(categorical_data)
    return(categorical_data) 

In [29]:
X = np.reshape(X_train.to_numpy(), (len(X_train), seq, 1))
y = create_categorical(y_train.to_numpy(), len(set(total_words)))

16560
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [13]:
X_val = np.reshape(X_val.to_numpy(), (len(X_val), seq, 1))
y_val = create_categorical(y_val.to_numpy(), len(set(total_words)))

11040
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [28]:
print(y.shape)

(16560, 1011)



### Plot Generation of Validation vs Train

In [0]:
def plot_history(histories, key='categorical_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')
  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()
  plt.xlim([0,max(history.epoch)])

# Bidirectional LSTM Model Creation

In [0]:
def createModel(X,y):
  model = Sequential()
  model.add(Bidirectional(LSTM(512, return_sequences=True),input_shape=(X.shape[1], X.shape[2])))
  model.add(Dropout(0.2))
  model.add(Bidirectional(LSTM(512)))
  model.add(Dropout(0.2))
  model.add(Dense(y.shape[1], activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', 'categorical_crossentropy'])
  model.summary()
  return model

Create Bidirectional Model

In [31]:
model = createModel(X, y)




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 3, 1024)           2105344   
_________________________________________________________________
dropout_9 (Dropout)          (None, 3, 1024)           0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 1024)              6295552   
_________________________________________________________________
dropout_10 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1011)              1036275   
Total params: 9,437,171
Trainable params: 9,437,171
Non-trainable params: 0
_________________________________________________________

Fit the model

In [24]:
histories=model.fit(X, y, epochs=100, batch_size=128, validation_data=(X_val, y_val))

AttributeError: ignored

In [0]:
model.save('my_model.h5')

In [0]:
# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3, 512)            1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 512)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1011)              518643    
Total params: 3,670,515
Trainable params: 3,670,515
Non-trainable params: 0
_

In [0]:
plot_history([('baseline', histories)])

In [0]:
word_dict

In [0]:
def generateTweet(fileN):
  start = np.random.randint(0, len(initial_word)-1)
  seed_text = initial_word[start]
  print(seed_text)
  seed = [word_dict[word] for word in seed_text]
  #print(seed)
  generatedTextList= seed_text
  flag = True
  #print(generatedTextList)
  count=0

  while (flag):
    x = np.reshape(seed, (1, len(seed), 1))
    x = x / float(len(set(total_words)))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
  #             result = datay_word[index]
    result = int_to_word[index]
    generatedTextList.append(result)
    seq_in = [int_to_word[value] for value in seed]
  #         sys.stdout.write(result+ " ")
  #         print(result+ " ",end="")
    seed.append(index)
    seed = seed[1:len(seed)]
    if result == "\n" or count==15:
        flag = False
    count+=1

  print(generatedTextList, file = fileN)
  print(generatedTextList)

In [0]:
fileN=open("generatedText.txt","w")
for _ in range(500):
  generateTweet(fileN)
fileN.close()

In [0]:
import numpy as np
from sklearn.model_selection import KFold

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4])
kf = KFold(n_splits=2)
kf.get_n_splits(X)